In [2]:
import heapq
import math

# Heuristic functions

def manhattan(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def euclidean(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def diagonal(a, b):
    return max(abs(a[0] - b[0]), abs(a[1] - b[1]))



# Pathfinding Class

class Pathfinding:
    def __init__(self, grid, start, goal, heuristic="manhattan"):
        self.grid = grid
        self.start = start
        self.goal = goal
        self.heuristic = {
            "manhattan": manhattan,
            "euclidean": euclidean,
            "diagonal": diagonal
        }[heuristic]
        self.rows = len(grid)
        self.cols = len(grid[0])

    def get_neighbors(self, node):
        directions = [(1,0), (-1,0), (0,1), (0,-1)]  # only 4 moves
        neighbors = []
        for dr, dc in directions:
            r, c = node[0] + dr, node[1] + dc
            if 0 <= r < self.rows and 0 <= c < self.cols and self.grid[r][c] != 1:
                neighbors.append((r, c))
        return neighbors

    def reconstruct_path(self, came_from):
        node = self.goal
        path = []
        while node in came_from:
            path.append(node)
            node = came_from[node]
        path.append(self.start)
        return path[::-1]

 
    # Greedy Best-First Search

    def greedy_best_first(self):
        open_list = []
        heapq.heappush(open_list, (self.heuristic(self.start, self.goal), self.start))
        came_from = {}
        visited = set([self.start])

        while open_list:
            _, current = heapq.heappop(open_list)

            if current == self.goal:
                return self.reconstruct_path(came_from), visited

            for neighbor in self.get_neighbors(current):
                if neighbor not in visited:
                    visited.add(neighbor)
                    came_from[neighbor] = current
                    heapq.heappush(open_list, (self.heuristic(neighbor, self.goal), neighbor))

        return None, visited

    
    # A* Search
   
    def a_star(self):
        open_list = []
        g_score = {self.start: 0}
        f_score = {self.start: self.heuristic(self.start, self.goal)}
        heapq.heappush(open_list, (f_score[self.start], self.start))
        came_from = {}
        visited = set([self.start])

        while open_list:
            _, current = heapq.heappop(open_list)

            if current == self.goal:
                return self.reconstruct_path(came_from), visited

            for neighbor in self.get_neighbors(current):
                tentative_g = g_score[current] + 1
                if tentative_g < g_score.get(neighbor, float("inf")):
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score[neighbor] = tentative_g + self.heuristic(neighbor, self.goal)
                    heapq.heappush(open_list, (f_score[neighbor], neighbor))
                    visited.add(neighbor)

        return None, visited



# Example Usage

if __name__ == "__main__":
   
    grid = [
        [0,0,0,1,0],
        [1,1,0,1,0],
        [0,0,0,1,0],
        [1,1,0,1,1],
        [0,0,0,0,0]
    ]
 # Grid: 0 = open, 1 = wall, S = start, G = goal
    start = (0,0)   
    goal = (4,4)    

    for h in ["manhattan", "euclidean", "diagonal"]:
        print(f"\n--- Using {h} heuristic ---")
        pf = Pathfinding(grid, start, goal, heuristic=h)

        path, visited = pf.greedy_best_first()
        print("Greedy Best-First Path:", path)
        print("Length:", len(path) if path else None, "Nodes explored:", len(visited))

        path, visited = pf.a_star()
        print("A* Path:", path)
        print("Length:", len(path) if path else None, "Nodes explored:", len(visited))


--- Using manhattan heuristic ---
Greedy Best-First Path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4)]
Length: 9 Nodes explored: 11
A* Path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4)]
Length: 9 Nodes explored: 11

--- Using euclidean heuristic ---
Greedy Best-First Path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4)]
Length: 9 Nodes explored: 11
A* Path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4)]
Length: 9 Nodes explored: 11

--- Using diagonal heuristic ---
Greedy Best-First Path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4)]
Length: 9 Nodes explored: 11
A* Path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4)]
Length: 9 Nodes explored: 12
